In [1]:
# import dependencies
# import pandas as pd
# import numpy as np
import random
import json


In [2]:
# select desired state
state = 'missouri'

# access json file with data to be sonified for avaliable states
stateData = json.load(open('../resources/stateData.json'))

# list to hold probabilities for which demographic a new case will belong to
infectProbs = []
# list to hold probability that an infection will result in death for each demographic
deathProbs = []

# populate lists above
for data in stateData[state]:
    infectProbs.append(stateData[state][data]['chance_of_infection'])
    deathProbs.append(stateData[state][data]['chance_of_death'])


In [3]:
# function to determine how many cases correspond to a particular threshold
def nthThresh(n,thresh):
    return 100 * n + thresh

# function to adjust spreadProbs weights and calculate r0 for corresponding threshold
def newThresh():
    global n
    global thresh
    if n == 1:
        spreadProbs[3] -= 1
        spreadProbs[2] += 1
    elif n == 2:
        spreadProbs[3] -= 1
        spreadProbs[1] += 1
    elif n == 13:
        spreadProbs[2] -= 1
        spreadProbs[0] += 1
    elif n == 14:
        spreadProbs[1] -= 1
        spreadProbs[0] += 1
    elif n != 0 and n < 13:
        spreadProbs[3] -= 1
        spreadProbs[0] += 1
    r0 = (spreadProbs[1] + 2*spreadProbs[2] + 3*spreadProbs[3]) / sum(spreadProbs)
    # calculate new threshold
    thresh = nthThresh(n, thresh)
    # add one to threshold n value
    n += 1
    
def newTrans():
    global trans
    t = random.randint(0,12)-6
    if t != 0:
        trans += t
    else:
        trans += random.choice([-1, 1])

#     t = random.randint(1,12)
#     if trans+t > 12:
#         trans -= t
#     elif trans-t < 12:
#         trans += t
        
        
        

# function to select rescaled incubation period as a delay in milliseconds.
def incubate(n):
    if n == 1:  
        sub = 3
        return random.randint(0, sub*40)*2000/sub
    elif n == 2:
        sub = random.choice([3,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 3:
        sub = random.choice([6,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 4:
        sub = random.choice([6,8])
        return random.randint(0, sub*40)*2000/sub
    elif n == 5:
        sub = random.choice([6,8,5])
        return random.randint(0, sub*40)*2000/sub
    else:
        sub = random.choice([6,8,5,7])
        return random.randint(0, sub*40)*2000/sub

# function to select a value based on a probability distribution array
def select(probsArray):
    return random.choices(list(range(len(probsArray))),weights=probsArray)[0]

# function to decide between one and zero based on probability that 1 will be chosen
def decide(prob):
    return random.choices([0,1], weights=[100-prob, prob])[0]

# function to generate midi notes
def midi(trans, death):
    note = random.choice(midiNotes)
    return note + trans - death

In [4]:
# weighted probabilities for #people the current case can spread infection to
# (from left to right, values refer to weights for 0, 1, 2, and 3 new infections)
spreadProbs = [0,0,0,12]
# symptomatic cases that have occured
sympCount = 0
# deaths that have occured
deathCount = 0
# case count threshold for when new r0 shoud be calculated
thresh = 0
# nth threshold
n = 1
# infection rate
r0 = 3
# transposition in semitones
trans = 0
# cases remaining (one case to start it off)
times = []
# which case
case = 0
# array of midi notes
midiNotes = [48, 51, 55, 58, 60, 62, 63, 65, 67, 68, 70, 74, 77, 80]
# dictionary with all data
weathering = {
    'time': [],
    'demo': [],
    'note': [],
    'death': [],
    'n': [],
}


In [5]:
times.append(incubate(n))

while len(times) > 0:
    
    # determine how much time has elapse since beginning of piece
    now = min(times)
    
    # if symptomatic add case info to weathering
    if decide(40) == 0:

        # add current case to total # of cases
        # (only sympotmatic cases adjust threshold)
        sympCount += 1
        
        if sympCount % 50 == 0:
            newTrans()

         # if case count is at or over threshold number
        if sympCount > nthThresh(n, thresh):
            newThresh()
            
        weathering['time'].append(now)

        # select which demographic the current case belongs to
        demo = select(infectProbs)
        weathering['demo'].append(demo)

        # decide/record if case results in death or not
        death = decide(deathProbs[demo])
        weathering['death'].append(death)

        # select which note
        weathering['note'].append(midi(trans, death))

        weathering['n'].append(n)
    
   

    # decide how many people the current case infects and add them to more
    for i in range(select(spreadProbs)):
        
        # select incubation period (delay time)
        delay = incubate(n)
        
        # determine time elapsed since beginning and append to times list
        time = now + delay
        times.append(time)

    # remove current case from timestamps of remaining cases
    times.pop(times.index(now))


In [6]:
result = sorted(list(zip(weathering['time'], weathering['demo'], weathering['note'], weathering['death'], weathering['n'])))

len(result)


11364

In [7]:
result

[(54666.66666666667, 1, 48, 0, 1),
 (64666.66666666667, 5, 74, 0, 1),
 (68000.0, 5, 51, 0, 1),
 (72000.0, 0, 63, 0, 1),
 (77333.33333333333, 2, 62, 0, 1),
 (84666.66666666666, 3, 67, 0, 1),
 (84666.66666666667, 3, 47, 1, 1),
 (87333.33333333334, 5, 51, 0, 1),
 (94000.0, 3, 74, 0, 1),
 (98666.66666666667, 0, 58, 0, 1),
 (100000.0, 1, 67, 0, 1),
 (101333.33333333334, 1, 55, 0, 1),
 (104000.0, 5, 68, 0, 1),
 (104000.00000000001, 3, 51, 0, 1),
 (105333.33333333333, 0, 70, 0, 1),
 (106000.0, 1, 58, 0, 1),
 (112000.0, 2, 55, 0, 1),
 (112000.00000000001, 2, 60, 0, 1),
 (112666.66666666666, 3, 63, 0, 1),
 (114000.0, 3, 60, 0, 1),
 (114000.00000000001, 1, 77, 0, 1),
 (118666.66666666666, 3, 74, 0, 1),
 (122000.0, 0, 60, 0, 1),
 (124000.0, 5, 70, 0, 1),
 (124666.66666666667, 5, 48, 0, 1),
 (126000.0, 1, 80, 0, 1),
 (126000.0, 5, 58, 0, 1),
 (128666.66666666669, 3, 63, 0, 1),
 (130666.66666666666, 5, 62, 0, 1),
 (132000.0, 0, 58, 0, 1),
 (132666.6666666667, 1, 62, 1, 1),
 (133333.3333333333, 5, 6

In [8]:
# for i, r in enumerate(result):
#     print(f'{i}: {r[4]}')



In [9]:
meep = float(round(result[3][0],6))
moop = float(round(result[4][0],6))
print(meep)
print(moop)

moop - meep


72000.0
77333.333333


5333.333333000002

In [10]:
timing = {}
ms = 0
infections = -1

for case in result:
    
    time = round(case[0], 6)

    if time != ms:
        infections += 1
        timing[infections] = {'demoNotes': [case[1], case[2], case[3]]}
        timing[infections]['delay'] = round(time - ms, 6)
        ms = time

    else:
        for i in range(1,4):
            timing[infections]['demoNotes'].append(case[i])


In [11]:
timing



{0: {'demoNotes': [1, 48, 0], 'delay': 54666.666667},
 1: {'demoNotes': [5, 74, 0], 'delay': 10000.0},
 2: {'demoNotes': [5, 51, 0], 'delay': 3333.333333},
 3: {'demoNotes': [0, 63, 0], 'delay': 4000.0},
 4: {'demoNotes': [2, 62, 0], 'delay': 5333.333333},
 5: {'demoNotes': [3, 67, 0, 3, 47, 1], 'delay': 7333.333334},
 6: {'demoNotes': [5, 51, 0], 'delay': 2666.666666},
 7: {'demoNotes': [3, 74, 0], 'delay': 6666.666667},
 8: {'demoNotes': [0, 58, 0], 'delay': 4666.666667},
 9: {'demoNotes': [1, 67, 0], 'delay': 1333.333333},
 10: {'demoNotes': [1, 55, 0], 'delay': 1333.333333},
 11: {'demoNotes': [5, 68, 0, 3, 51, 0], 'delay': 2666.666667},
 12: {'demoNotes': [0, 70, 0], 'delay': 1333.333333},
 13: {'demoNotes': [1, 58, 0], 'delay': 666.666667},
 14: {'demoNotes': [2, 55, 0, 2, 60, 0], 'delay': 6000.0},
 15: {'demoNotes': [3, 63, 0], 'delay': 666.666667},
 16: {'demoNotes': [3, 60, 0, 1, 77, 0], 'delay': 1333.333333},
 17: {'demoNotes': [3, 74, 0], 'delay': 4666.666667},
 18: {'demoNo

In [12]:
with open("../resources/weathering.json", "w") as outfile:
    json.dump(timing, outfile)
    